# Libraries

In [1]:
import requests, bs4
from urllib.parse import urlsplit
from fuzzywuzzy import fuzz
import re

D:\Users\Ritvik\Anaconda3\envs\datascience\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Scraping

In [2]:
EMAIL = re.compile(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+')
CSS = re.compile(r'<style(.|\n)*?>(.|\n)*?</style>')
JS = re.compile(r'<script(.|\n)*?>(.|\n)*?</script>')
HTML = re.compile(r'<(.|\n)*?>')

In [3]:
url = 'http://www.miet.ac.in'

In [4]:
emails = set()
unscraped = [url]
scraped = []
while len(unscraped):
    url = unscraped.pop(0)
    scraped.append(url)
    print('Crawling', url)
    parts = urlsplit(url)
    base_url = "{0.scheme}://{0.netloc}".format(parts)
    if '/' in parts.path:
        path = url[:url.rfind('/')+1]
    else:
        path = url
    try:
        res = requests.get(url)
    except Exception as e:
        print(e)
    
    if res.status_code == requests.codes.ok:
        contents  = res.text
        contents = CSS.sub(' ', contents)
        contents = JS.sub(' ', contents)
        contents = HTML.sub(' ', contents)
        contents = re.sub(r"\s+", ' ', contents)
        new_emails = set(EMAIL.findall(contents))
        emails = emails |new_emails
        
        
        ressoup = bs4.BeautifulSoup(res.text, 'lxml')
        for anchor in ressoup.find_all("a"):
            _ = anchor.getText()
            if "href" in anchor.attrs:
                link = anchor.attrs["href"]
            else:
                link = ''

            if link.startswith('/'):
                link = base_url + link
            if fuzz.token_set_ratio(_, 'about us') > 80 or fuzz.token_set_ratio(_, 'contact us') > 80:
                if not link in unscraped and not link in scraped:
                    print('Adding:', _)
                    unscraped.append(link)
                             
    else:
        print('Something went wrong')

Crawling http://www.miet.ac.in
Adding: About
Adding: Placement Office Contact
Adding: Contact Us
Adding: Contact Us
Adding:  Placement Office Contact
Crawling #
Invalid URL '#': No schema supplied. Perhaps you meant http://#?
Crawling https://miet.ac.in/contact-us
Crawling contact
Invalid URL 'contact': No schema supplied. Perhaps you meant http://contact?
Crawling https://miet.ac.in/contact
Crawling contact-us
Invalid URL 'contact-us': No schema supplied. Perhaps you meant http://contact-us?


In [5]:
print(emails)

{'placement@miet.ac.in', 'contact@miet.ac.in'}
